In [74]:
import pandas as pd

In [75]:
root = "/home/sihartist/Desktop/"

In [76]:
# Load the credit card transaction data
df = pd.read_csv('/home/sihartist/Desktop/fraud-detection/algo/dataset/data_balanced.csv')
df = df.iloc[:,1:]

df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,isFlaggedFraud,isFraud
0,139,265803.35,0.00,0.00,751669.39,1017472.74,0,1,0,0,0,0,0
1,589,11278.28,0.00,0.00,220970.84,232249.12,0,1,0,0,0,0,0
2,284,16363.06,35966.16,19603.10,0.00,0.00,0,0,0,1,0,0,0
3,37,107223.31,20752629.71,20859853.02,804594.62,571711.22,1,0,0,0,0,0,0
4,287,3678.25,0.00,0.00,0.00,0.00,0,0,0,1,0,0,0


In [78]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Split the data into training and test sets
X = df.drop('isFraud', axis=1).values
y = df['isFraud'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for input into the CNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))
print(X_train.shape)
# Build the deep CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

(13140, 12)
Epoch 1/10
411/411 [==============================] - 1s 2ms/step - loss: 0.3070 - accuracy: 0.8690
Epoch 2/10
411/411 [==============================] - 1s 2ms/step - loss: 0.2188 - accuracy: 0.9017
Epoch 3/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1929 - accuracy: 0.9128
Epoch 4/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1836 - accuracy: 0.9202
Epoch 5/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1781 - accuracy: 0.9218
Epoch 6/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1704 - accuracy: 0.9259
Epoch 7/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1661 - accuracy: 0.9261
Epoch 8/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1613 - accuracy: 0.9317
Epoch 9/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1599 - accuracy: 0.9328
Epoch 10/10
411/411 [==============================] - 1s 2ms/step - loss: 0.1535 - accu

In [79]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

103/103 [==============================] - 0s 2ms/step - loss: 0.1470 - accuracy: 0.9352
Test Accuracy: 0.9351795315742493


In [80]:
import pickle

filename = root + "fraud-detection/algo/models/cnn.pkl"
pickle.dump(model, open(filename, "wb"))

INFO:tensorflow:Assets written to: ram://3d7d20e2-7461-4cd8-9166-5b6dc55ff62e/assets


INFO:tensorflow:Assets written to: ram://3d7d20e2-7461-4cd8-9166-5b6dc55ff62e/assets


In [81]:
loaded_model = pickle.load(open(filename, "rb"))
test_loss, test_acc = loaded_model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

103/103 [==============================] - 0s 2ms/step - loss: 0.1470 - accuracy: 0.9352
Test Accuracy: 0.9351795315742493


In [91]:
test=[139, 265803.35, 0.00, 0.00, 751669.39, 1017472.74, 0, 1, 0, 0, 0 ,0]
test1=[319, 1033878.67, 1033878.67, 0.0, 0.0, 0.0, 0, 0, 0, 0, 1, 0]

to_predict1 = np.array(test).reshape(1,12)
to_predict2 = np.array(test1).reshape(1,12)

result = loaded_model.predict(to_predict1)
print(result)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_7/conv1d_8/Conv1D/ExpandDims' defined at (most recent call last):
    File "/home/sihartist/miniconda3/envs/my-env/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/sihartist/miniconda3/envs/my-env/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/sihartist/miniconda3/envs/my-env/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/sihartist/miniconda3/envs/my-env/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/sihartist/miniconda3/envs/my-env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/sihartist/miniconda3/envs/my-env/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, 